## Importação das Bibiliotecas

In [54]:
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import pyspark.sql.functions as F

In [55]:
# Crie um SparkSession
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [56]:
#Path dos arquivos

caminho_arquivo_2022 = r'E:\Estudos\SQL\Datasets\ENEM\2022\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'
caminho_arquivo_2021 = r'E:\Estudos\SQL\Datasets\ENEM\2021\microdados_enem_2021\DADOS\MICRODADOS_ENEM_2021.csv'
caminho_arquivo_2020 = r'E:\Estudos\SQL\Datasets\ENEM\2020\microdados_enem_2020\DADOS\MICRODADOS_ENEM_2020.csv'
caminho_arquivo_2019 = r'E:\Estudos\SQL\Datasets\ENEM\2019\microdados_enem_2019\DADOS\MICRODADOS_ENEM_2019.csv'

In [76]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = [F.col(df.columns[i]) for i in range(0, 12)] + [F.col(df.columns[i]) for i in range(31, 34)] + [F.col(df.columns[39])]
    return df.select(colunas_selecionadas)


In [77]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfDadosParticipantes22 = lerCSV(spark, caminho_arquivo_2022)
dfDadosParticipantes21 = lerCSV(spark, caminho_arquivo_2021)
dfDadosParticipantes20 = lerCSV(spark, caminho_arquivo_2020)
dfDadosParticipantes19 = lerCSV(spark, caminho_arquivo_2019)



In [78]:
#Unir os DF em um só 

dfDadosParticipantes = dfDadosParticipantes22.union(dfDadosParticipantes21).union(dfDadosParticipantes20).union(dfDadosParticipantes19)


In [79]:
dfDadosParticipantes.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------+----------+----------+---------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|TP_LINGUA|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------+----------+----------+---------+
|210057943671|  2022|             14|      M|              2|          2|               1|              1|              2|        1|     null|           0|      null|      null|      null|        0|
|210057516120|  2022|             14|      M|              2|          1|               1|              1|             16|        1|     null|           0|      null|      null|      null|        0|
|2100

In [80]:
dfPreencher = dfDadosParticipantes.fillna(-2)  # Preenche com -2(sem ref), por exemplo


In [81]:
dfPreencher.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------+----------+----------+---------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|TP_LINGUA|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------+----------+----------+---------+
|210057943671|  2022|             14|      M|              2|          2|               1|              1|              2|        1|       -2|           0|      -2.0|      -2.0|      -2.0|        0|
|210057516120|  2022|             14|      M|              2|          1|               1|              1|             16|        1|       -2|           0|      -2.0|      -2.0|      -2.0|        0|
|2100

# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER

In [82]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "Participantes"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [83]:
# Nome da nova tabela que será criada
nomeTabela = 'Participantes'

In [84]:
#Conectando via jdbc
#No método option, foi necessário passar os argumentos ssl e trustservercertificate como true, por se tratar de um banco local sem SSL.
try:
  jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()
  
  jdbcDF.show(40)
  print(jdbcDF.count())
except ValueError as error :
    print("Connector write failed", error)


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|CO_PROVA_CN|CO_PROVA_CH|CO_PROVA_LC|CO_PROVA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|TP_LINGUA|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|
+------------+------+-------

In [85]:
# Configurações do banco de dados
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = "DROP TABLE dbo.Participantes"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [86]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [87]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela Participantes criada com sucesso no banco de dados.


In [88]:
spark.stop()